# Models overview

In this notebook, we explore the main building block of Hezar which is the "models". In Hezar, models are the typical PyTorch modules with some extra features for loading, saving, exporting, etc. Lets dive into some of the most important ones!

## Build model

Like any other package, you can import any model from `hezar.models` that you want.

In [1]:
from hezar.models import BertLM, BertLMConfig

bert = BertLM(BertLMConfig())
print(bert)

/home/aryan/Applications/miniconda3/envs/main/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

You can also configure the architecture by changing the properties in a model's config like so:

In [2]:
config = BertLMConfig(num_hidden_layers=8, num_attention_heads=8)
bert = BertLM(config)
bert

BertLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

#### Build a model with registry name

Hezar uses a global container for all of its modules called a `registry`. Registries are simple Python dictionaries that store a module class and its config class under a unique key. An example models' registry has the following format.

```python
{
    'bert_lm': {
        'model_class': hezar.models.language_modeling.bert.bert_lm.BertLM,
        'config_class': hezar.models.language_modeling.bert.bert_lm_config.BertLMConfig
    },
}
```

As you can see, above is a sample models registry with one namespace called `bert_lm` that is a key to another dictionary that stores the model class and config class. 


So lets use our `models_registry` to build a model:

In [3]:
from hezar.registry import models_registry

config = models_registry["bert_lm"]["config_class"]()
model = models_registry["bert_lm"]["model_class"](config)
model

BertLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

To be fair, this is not the cleanest way to build a model so let us introduce the `builders`! builders are some tiny functions to build a module using the registry name and pass custom config parameters as keyword arguments. Take a look at the below example:

In [4]:
from hezar.builders import build_model

bert = build_model("bert_lm", num_hidden_layers=8, num_attention_heads=8)  # pass in config parameters as keyword arguments to the build function
bert

BertLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

#### But why?
So you might ask, why would you store everything inside a registry when you can normally import anything you want and build instances?

To answer this take a look at the below examples:

In [5]:
# Import everything
from hezar import build_model, BertLM, BertLMConfig
config = BertLMConfig(num_hidden_layers=8, num_attention_heads=8)
bert = BertLM(config)

# Use builder
bert = build_model("bert_lm", num_hidden_layers=8, num_attention_heads=8)

The above code examples are not way too different. In order to build a model with `build_model()` you have to know its registry name and build it with one line of code. But the downside is that your IDE/editor cannot give you proper suggestions as it does not know what type of a model it's returning! So it's actually up to you to pick the method you feel comfortable with, but the purpose of the registries is not just being able to build modules using their names! But in fact:
1. Registries store every registered module in a single, global container so that accessing them is way simpler.
2. Configuring a module type in a config file is better be of a string type. For example if you want to specify the type of the dataset, optimizer, etc. in a train config, you can just put its registry name and the rest is taken care of! As we go further you'll see more of these usecases.
3. In Hezar, the base classes like `hezar.models.Model` or `hezar.configs.Config` or `hezar.preprocessors.Tokenizer` have a class method called `load()` that can load a fully functional module from the Hub. This method first downloads the config file and then builds the proper object using the `name` parameter which is, you guessed it!, the registry name of that module! 

## Load a model from Hub

Every model in Hezar, can be pushed to the Hub or downloaded from it. 

Loading a model from Hub is as easy as:

In [6]:
from hezar import Model

bert = Model.load("hezar-ai/bert-base-fa")
bert

BertLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

## Add your own model

In order to add a model to Hezar or perhaps write a custom model with Hezar's Model functionalities follow the steps below:

In [7]:
from dataclasses import dataclass

from torch import Tensor, nn

from hezar import Model, ModelConfig, register_model


@dataclass
class PerceptronConfig(ModelConfig):
    name: str = "perceptron"
    input_shape: int = 4
    output_shape: int = 2

@register_model("preceptron", config_class=PerceptronConfig)
class Perceptron(Model):
    """
    A simple MLP
    """

    def __init__(self, config, **kwargs):
        super().__init__(config, **kwargs)
        self.nn = nn.Linear(in_features=self.config.input_shape, out_features=self.config.output_shape)

    def forward(self, inputs: list, **kwargs):
        inputs = Tensor(inputs).reshape(1, -1)
        x = self.nn(inputs)
        return x


In [8]:
model = Perceptron(PerceptronConfig())
inputs = [1, 2, 3, 4]
model.predict(inputs)

tensor([[1.6096, 0.4799]])

### Save model

In [9]:
model.save("perceptron")

'perceptron/model.pt'

### Push to Hub

In [10]:
model.push_to_hub("hezar-ai/perceptron", private=True)

INFO: Prepared repo `https://huggingface.co/hezar-ai/perceptron`. Starting push process...
INFO: Uploaded `perceptron` config to `hezar-ai/perceptron/` as `model_config.yaml`
model.pt: 100%|██████████| 1.05k/1.05k [00:02<00:00, 471B/s]  
Upload 1 LFS files: 100%|██████████| 1/1 [00:02<00:00,  2.25s/it]
INFO: Uploaded model files to `hezar-ai/perceptron`
